In [1]:
from math import exp, sqrt
import matplotlib.pyplot as plt
import random
import sys
import pandas as pd
import numpy as np
import os,errno
from mpl_toolkits.mplot3d import Axes3D
from scipy.sparse import csgraph
from numpy import linalg as LA

# K-means Clustering 

In [33]:
def centroid_init_random(data,kCluster,num_data):
    
    clusters = [[[None] for i in range(0)] for j in range(kCluster)]
    
    #initail the cluster centroids with random samples from data
    mu = data[random.sample(range(0,num_data-1), kCluster)]
    
    # randomly assign data to clusters  
    for i in range(num_data):
        k= random.randint(0,kCluster-1)
        clusters[k].append(data[i])
        
    return mu, clusters

In [34]:
def centroid_init_random_cluster_mean(data,kCluster,num_data):
    
    #initail the cluster centroids with random samples from data
    mu = data[random.sample(range(0,num_data-1), kCluster)]
    
    clusters = [[[None] for i in range(0)] for j in range(kCluster)]
    
    # randomly assign data to clusters and calculte another mu  
    for i in range(num_data):
        k= random.randint(0,kCluster-1)
        clusters[k].append(data[i])
    #calculate mu from randomly assigned clusters
    for k in range(kCluster): 
        mu[k] = np.mean(clusters[k],axis=0)
        
    return mu, clusters

In [38]:
def centroid_init_kmeans_plusplus(data, kCluster,num_data):
    
    mu = data[random.sample(range(0,num_data-1),1)]
    mu= mu.tolist()
    while len(mu)<kCluster:
        dist_center = np.array([min([np.linalg.norm(x-c)**2 for c in mu]) for x in data])
        probs = dist_center/dist_center.sum()
        cumprobs = probs.cumsum()
        r = random.random()
        ind = np.where(cumprobs >= r)[0][0]
        mu.append(data[ind].tolist())
        
    clusters = [[[None] for i in range(0)] for j in range(kCluster)]
    # randomly assign data to clusters  
    for i in range(num_data):
        k= random.randint(0,kCluster-1)
        clusters[k].append(data[i])
    
    return np.array(mu), clusters

In [44]:
def kmeans(data, kCluster,num_data, init_flag=0):
    
    if init_flag==0:
        mu,clusters = centroid_init_random(data,kCluster,num_data)
    elif init_flag==1:
        mu,clusters = centroid_init_random_cluster_mean(data,kCluster,num_data)
    elif init_flag==2:
        mu,clusters= centroid_init_kmeans_plusplus(data, kCluster,num_data)
        
                                             
                                             
    #set path for saving the plots after each iteration    
    iteration = 0
    path ="plots/Kmeans_on_"+dataset_name+"_dataset"+"_"+str(kCluster)+"clusters"
    #make directory using above path
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    # intialize an array of colors for coloring different clusters with different colors
    color = ["b","g","m","r","k"]
    for it in range (100) :
    
        
        #plot the clusters with different color and show centroid each iteration 
        for k in range (kCluster):
            plt.plot(np.array(clusters[k])[:,0],np.array(clusters[k])[:,1],color[k]+"o", alpha = 0.8)
            plt.plot(mu[k,0],mu[k,1],"k*", markersize=12, alpha = 1)

        clusters = [[[None] for i in range(0)] for j in range(kCluster)]

        for i in range (num_data) :
            min_dist = 1000000
            nearestK = - 1
            for k in range (kCluster) :
                    temp_dist = np.linalg.norm( data[i] - mu[k])
                    if (temp_dist < min_dist) :
                        min_dist = temp_dist
                        nearestK = k
                
            clusters[nearestK].append (data[i])

    
        plt.title("Kmeans"+" iter " +str(it))
        plt.axis('equal')
        plt.savefig(path+"/Kmeans_"+ str(it)+".png", bbox_inches="tight")
        plt.clf()
        newMu = np.zeros((kCluster,2))
        # calculate mean
        for k in range (kCluster) :
            newMu[k] = np.mean(clusters[k],axis =0) 
        if ( sqrt(np.linalg.norm(newMu-mu)) < 0.00001):
            iteration = it
            break
        mu = newMu.copy()
    
    print ("converged after ", iteration, " interation(s)")


In [50]:
fileName = input("Enter file name:")
dataset_name= fileName[:4]
kCluster = int(input('Enter the K number of clusters :'))
data = pd.read_csv(fileName,header=None)
data = np.array(data)
num_data = data.shape[0]
kmeans(data, kCluster,num_data, 2)

Enter file name:circle.txt
Enter the K number of clusters :4
converged after  13  interation(s)
